In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
# Using .getOrCreate will only create a new spark session once. 
# Subsequent calls to that method will re-use the existing session.

In [3]:
import multiprocessing

nprocs = multiprocessing.cpu_count()
nprocs

4

In [4]:
import pyspark.sql.functions as F

While spark dataframes might superficially look like pandas dataframes, and even share some of the same methods and syntax, it is important to keep in mind they are 2 seperate types of objects, and, while spark and pandas code might look superficially similar, it tends to be semantically very different.

In [5]:
import pandas as pd
import numpy as np

In [6]:
np.random.seed(456) # create pandas df to convert to a spark df

pandas_dataframe = pd.DataFrame(dict(n = np.arange(20), group = np.random.choice(list('abc'), 20)))
pandas_dataframe

,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


In [7]:
df = spark.createDataFrame(pandas_dataframe)
df #spark is lazy, in that it won't show us values until it has to

DataFrame[n: bigint, group: string]

In [8]:
df.show(5) # showing the Spark df

+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



In [10]:
df.describe().show()

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



In [11]:
from pydataset import data

mpg = spark.createDataFrame(data('mpg'))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



#### .select lets us specify what data we want to see in the resulting dataframe

In [13]:
mpg.select(mpg.hwy, mpg.cty, mpg.model).show()

+---+---+------------------+
|hwy|cty|             model|
+---+---+------------------+
| 29| 18|                a4|
| 29| 21|                a4|
| 31| 20|                a4|
| 30| 21|                a4|
| 26| 16|                a4|
| 26| 18|                a4|
| 27| 18|                a4|
| 26| 18|        a4 quattro|
| 25| 16|        a4 quattro|
| 28| 20|        a4 quattro|
| 27| 19|        a4 quattro|
| 25| 15|        a4 quattro|
| 25| 17|        a4 quattro|
| 25| 17|        a4 quattro|
| 25| 15|        a4 quattro|
| 24| 15|        a6 quattro|
| 25| 17|        a6 quattro|
| 23| 16|        a6 quattro|
| 20| 14|c1500 suburban 2wd|
| 15| 11|c1500 suburban 2wd|
+---+---+------------------+
only showing top 20 rows



In [16]:
mpg.select(mpg.hwy, mpg.hwy + 1).show(5)

+---+---------+
|hwy|(hwy + 1)|
+---+---------+
| 29|       30|
| 29|       30|
| 31|       32|
| 30|       31|
| 26|       27|
+---+---------+
only showing top 5 rows



#### use the .alias method to rename it

In [17]:
mpg.select(mpg.hwy.alias("highway_mileage")).show(5)

+---------------+
|highway_mileage|
+---------------+
|             29|
|             29|
|             31|
|             30|
|             26|
+---------------+
only showing top 5 rows



#### store column objects in variables and reference them

In [18]:
col1 = mpg.hwy.alias('higway_mileage')
col2 = (mpg.hwy / 2).alias("highway_mileage+halved")
mpg.select(col1, col2).show(5)

+--------------+----------------------+
|higway_mileage|highway_mileage+halved|
+--------------+----------------------+
|            29|                  14.5|
|            29|                  14.5|
|            31|                  15.5|
|            30|                  15.0|
|            26|                  13.0|
+--------------+----------------------+
only showing top 5 rows



#### create columns with the col and expr functions from pyspark.sql.functions module

In [19]:
from pyspark.sql.functions import col, expr

In [20]:
col('hwy')

Column<b'hwy'>

#### mix and match the syntax

In [24]:
avg_colum = (col('hwy') + col('cty')) /2

mpg.select(
    col('hwy').alias('highway_mileage'),
    mpg.cty.alias('city_mileage'),
    avg_column.alias('avg-mileage'),
).show(5)

+---------------+------------+-----------+
|highway_mileage|city_mileage|avg-mileage|
+---------------+------------+-----------+
|             29|          18|       23.5|
|             29|          21|       25.0|
|             31|          20|       25.5|
|             30|          21|       25.5|
|             26|          16|       21.0|
+---------------+------------+-----------+
only showing top 5 rows



#### exp does everything col does and more. expr returns the same type of column object, but allows us to express manipulations to the column within the string that defines the column.

In [26]:
mpg.select(
    expr('hwy'), # same as col
    expr('hwy + 1'), #addition function
    expr('hwy AS highway_mileage'), #using an alias
    expr('hwy + 1 AS highway_incremented') #combo of previous 2
).show(5)

+---+---------+---------------+-------------------+
|hwy|(hwy + 1)|highway_mileage|highway_incremented|
+---+---------+---------------+-------------------+
| 29|       30|             29|                 30|
| 29|       30|             29|                 30|
| 31|       32|             31|                 32|
| 30|       31|             30|                 31|
| 26|       27|             26|                 27|
+---+---------+---------------+-------------------+
only showing top 5 rows



#### Identical:
- mpg.hwy.alias("highway"),
- col("hwy").alias("highway"),
- expr("hwy").alias("highway"),
- expr("hwy AS highway"),

#### spark SQL, which lets us write SQL queries against our spark dataframes. to start using spark SQL, we'll first "register" the table with spark

In [29]:
mpg.createOrReplaceTempView('mpg') #Spark SQL registers the table with Spark

In [33]:
spark.sql("""
SELECT hwy, cty, (hwy + cty) / 2 AS avg 
FROM mpg
""").show(5) #write SQL query against spark

+---+---+----+
|hwy|cty| avg|
+---+---+----+
| 29| 18|23.5|
| 29| 21|25.0|
| 31| 20|25.5|
| 30| 21|25.5|
| 26| 16|21.0|
+---+---+----+
only showing top 5 rows



In [35]:
mpg.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [36]:
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [37]:
#cast a variable
mpg.select(mpg.hwy.cast('string')).printSchema()

#f a value is not able to be converted, it will be replaced with null

root
 |-- hwy: string (nullable = true)



### Basic built-in functions

concat, sum, avg, min, max

Note that importing the sum function directly will override the built-in sum function. This means you will get an error if you try to sum a list of numbers, because sum will refernce the pyspark sum function, which works with pyspark dataframe columns, while the built-in sum function works with lists of numbers. The same holds true for the built in min and max functions.

In [38]:
from pyspark.sql.functions import * #Can also import specific functions

In [42]:
mpg.select(
    sum(mpg.hwy),
    avg(mpg.hwy).alias('Highway Average'),
    min(mpg.hwy),
    max(mpg.hwy)
).show()

+--------+-----------------+--------+--------+
|sum(hwy)|  Highway Average|min(hwy)|max(hwy)|
+--------+-----------------+--------+--------+
|    5485|23.44017094017094|      12|      44|
+--------+-----------------+--------+--------+



In [45]:
mpg.select(concat(mpg.manufacturer, mpg.model)).show(10)

+---------------------------+
|concat(manufacturer, model)|
+---------------------------+
|                     audia4|
|                     audia4|
|                     audia4|
|                     audia4|
|                     audia4|
|                     audia4|
|                     audia4|
|             audia4 quattro|
|             audia4 quattro|
|             audia4 quattro|
+---------------------------+
only showing top 10 rows



In [46]:
mpg.select(concat(mpg.cyl, lit('cylinders'))).show(5) #f-string literal

+----------------------+
|concat(cyl, cylinders)|
+----------------------+
|            4cylinders|
|            4cylinders|
|            4cylinders|
|            4cylinders|
|            6cylinders|
+----------------------+
only showing top 5 rows



In [47]:
textdf = spark.createDataFrame(
    pd.DataFrame(
    {
        "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
    }
    
    )
)

In [48]:
textdf.show(truncate = False) #uncut

+---------------------------------------------+
|address                                      |
+---------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |
|303 Pearl Pkwy, San Antonio, TX 78215        |
|1255 SW Loop 410, San Antonio, TX 78227      |
+---------------------------------------------+



In [49]:
textdf.show()

+--------------------+
|             address|
+--------------------+
|600 Navarro St st...|
|3130 Broadway St,...|
|303 Pearl Pkwy, S...|
|1255 SW Loop 410,...|
+--------------------+



#### The regexp_extract function lets us specify at least one capture group, and create a new column based on the contents of the capture group.

In [50]:
textdf.select(
    'address',
    regexp_extract('address', r"^(\d+)", 1).alias('street_no'),
    regexp_extract('address', r"^\d+\s([\w\s]+?),", 1).alias('street'),

).show(truncate = False)

+---------------------------------------------+---------+------------------+
|address                                      |street_no|street            |
+---------------------------------------------+---------+------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|600      |Navarro St ste 600|
|3130 Broadway St, San Antonio, TX 78209      |3130     |Broadway St       |
|303 Pearl Pkwy, San Antonio, TX 78215        |303      |Pearl Pkwy        |
|1255 SW Loop 410, San Antonio, TX 78227      |1255     |SW Loop 410       |
+---------------------------------------------+---------+------------------+



### .filter and .where

In [52]:
mpg.filter(mpg.cyl == 4).where(mpg['class'] == 'subcompact').show()

+------------+-----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|      model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+-----------+-----+----+---+----------+---+---+---+---+----------+
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 25| 32|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 23| 29|  p|subcompact|
|       honda|      civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|manual(m5)|  f| 26| 34|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|  auto(l5)|  f| 25| 36|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|  auto(l5)|  f| 24| 36|  c|subcompact|
|       honda|      civic|  2.0|2008|  4|manual(m6)|  f| 21| 29|  p|subcompact|
|     hyundai|    tiburon|  2.0|1999|  4

#### When and Otherwise

In [55]:
mpg.select(mpg.hwy, when(mpg.hwy > 25, 'good_mileage').otherwise('bad_mileage').alias('mpg_desc')).show(12)

+---+------------+
|hwy|    mpg_desc|
+---+------------+
| 29|good_mileage|
| 29|good_mileage|
| 31|good_mileage|
| 30|good_mileage|
| 26|good_mileage|
| 26|good_mileage|
| 27|good_mileage|
| 26|good_mileage|
| 25| bad_mileage|
| 28|good_mileage|
| 27|good_mileage|
| 25| bad_mileage|
+---+------------+
only showing top 12 rows



In [56]:
mpg.select(
    mpg.displ,
    (
        when(mpg.displ < 2, 'small')
        .when(mpg.displ < 3, 'medium')
        .otherwise('large')
        .alias('engine_size')
    ),
).show(10)

+-----+-----------+
|displ|engine_size|
+-----+-----------+
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
|  2.0|     medium|
|  2.8|     medium|
|  2.8|     medium|
|  3.1|      large|
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
+-----+-----------+
only showing top 10 rows



#### Sorting and Ordering

Spark lets us sort the rows in our dataframe by one or multiple columns with two methods: .sort, and .orderBy. .sort and .orderBy are aliases of each other and do the exact same thing. Like other methods we've seen, .sort takes in a Column object or a string that is the name of a column.

In [57]:
mpg.sort(mpg.hwy).show(8)

+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|manufacturer|              model|displ|year|cyl|     trans|drv|cty|hwy| fl| class|
+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|        jeep| grand cherokee 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|manual(m6)|  4|  9| 12|  e|pickup|
|       dodge|        durango 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|  dakota pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|   chevrolet|    k1500 tahoe 4wd|  5.3|2008|  8|  auto(l4)|  4| 11| 14|  e|   suv|
|        jeep| grand cherokee 4wd|  6.1|2008|  8|  auto(l5)|  4| 11| 14|  p|   suv|
|       dodge|        durango 4wd|  5.9|1999|  8|  auto(l4)|  4| 11| 15|  r|   suv|
+------------+-------------------+-----+----+---+----------+---+---+---+---+

In [59]:
#all 3 are the same

mpg.sort(mpg.hwy.desc())
mpg.sort(col('hwy').desc())
mpg.sort(desc('hwy')).show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|  volkswagen|     jetta|  1.9|1999|  4|manual(m5)|  f| 33| 44|  d|   compact|
|  volkswagen|new beetle|  1.9|1999|  4|manual(m5)|  f| 35| 44|  d|subcompact|
|  volkswagen|new beetle|  1.9|1999|  4|  auto(l4)|  f| 29| 41|  d|subcompact|
|      toyota|   corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|       honda|     civic|  1.8|2008|  4|  auto(l5)|  f| 24| 36|  c|subcompact|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
only showing top 5 rows



#### To specify sorting by multiple columns, we provide each column as a separate argument to .sort.

In [61]:
mpg.sort(desc('class'), mpg.cyl.asc(), col('hwy').desc()).show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-----+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-----+
|      subaru|      forester awd|  2.5|2008|  4|manual(m5)|  4| 20| 27|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|  auto(l4)|  4| 20| 26|  r|  suv|
|      subaru|      forester awd|  2.5|1999|  4|manual(m5)|  4| 18| 25|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|manual(m5)|  4| 19| 25|  p|  suv|
|      subaru|      forester awd|  2.5|1999|  4|  auto(l4)|  4| 18| 24|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|  auto(l4)|  4| 18| 23|  p|  suv|
|      toyota|       4runner 4wd|  2.7|1999|  4|  auto(l4)|  4| 16| 20|  r|  suv|
|      toyota|       4runner 4wd|  2.7|1999|  4|manual(m5)|  4| 15| 20|  r|  suv|
|        jeep|grand cherokee 4wd|  3.0|2008|  6|  auto(l5)|  4| 17| 22|  d|  suv|
|        jeep|gr

#### Grouping and Aggregating

To aggregate our data by group, we can use the .groupBy method. Like with .select, we can pass either Column objects or strings that are column names to .groupBy. All of the expressions below are equivalent.

In [65]:
mpg.groupBy(mpg.cyl)
mpg.groupBy(col('cyl'))
mpg.groupBy('cyl')

In [66]:
mpg.groupBy(mpg.cyl).agg(avg(mpg.cty), avg(mpg.hwy)).show()

+---+------------------+-----------------+
|cyl|          avg(cty)|         avg(hwy)|
+---+------------------+-----------------+
|  6| 16.21518987341772|22.82278481012658|
|  5|              20.5|            28.75|
|  8|12.571428571428571|17.62857142857143|
|  4|21.012345679012345|28.80246913580247|
+---+------------------+-----------------+



To group by multiple columns, pass each of the columns a a separate argument to .groupBy (Note that this is different from pandas, where we would need to pass a list)

In [67]:
mpg.groupBy('cyl', 'class').agg(avg(mpg.cty), avg(mpg.hwy)).show()

+---+----------+------------------+------------------+
|cyl|     class|          avg(cty)|          avg(hwy)|
+---+----------+------------------+------------------+
|  5|   compact|              21.0|              29.0|
|  5|subcompact|              20.0|              28.5|
|  6|subcompact|              17.0|24.714285714285715|
|  6|    pickup|              14.5|              17.9|
|  4|subcompact|22.857142857142858| 30.80952380952381|
|  8|       suv|12.131578947368421|16.789473684210527|
|  8|    pickup|              11.8|              15.8|
|  8|   midsize|              16.0|              24.0|
|  4|   midsize|              20.5|           29.1875|
|  8|   2seater|              15.4|              24.8|
|  6|   compact|16.923076923076923|25.307692307692307|
|  6|   minivan|              15.6|              22.2|
|  4|   compact|            21.375|          29.46875|
|  8|subcompact|              14.8|              21.6|
|  6|   midsize|17.782608695652176| 26.26086956521739|
|  4|   mi

In addition to .groupBy, we can use .rollup, which will do the same aggregations, but will also include the overall total

In [69]:
mpg.rollup('cyl').count().sort('cyl').show()

+----+-----+
| cyl|count|
+----+-----+
|null|  234|
|   4|   81|
|   5|    4|
|   6|   79|
|   8|   70|
+----+-----+



In [71]:
mpg.rollup('cyl').agg(expr('avg(hwy)')).sort('cyl').show()

+----+-----------------+
| cyl|         avg(hwy)|
+----+-----------------+
|null|23.44017094017094|
|   4|28.80246913580247|
|   5|            28.75|
|   6|22.82278481012658|
|   8|17.62857142857143|
+----+-----------------+



In [72]:
mpg.rollup('cyl', 'class').mean('hwy').sort(col('cyl'), col('class')).show()

+----+----------+------------------+
| cyl|     class|          avg(hwy)|
+----+----------+------------------+
|null|      null| 23.44017094017094|
|   4|      null| 28.80246913580247|
|   4|   compact|          29.46875|
|   4|   midsize|           29.1875|
|   4|   minivan|              24.0|
|   4|    pickup|20.666666666666668|
|   4|subcompact| 30.80952380952381|
|   4|       suv|             23.75|
|   5|      null|             28.75|
|   5|   compact|              29.0|
|   5|subcompact|              28.5|
|   6|      null| 22.82278481012658|
|   6|   compact|25.307692307692307|
|   6|   midsize| 26.26086956521739|
|   6|   minivan|              22.2|
|   6|    pickup|              17.9|
|   6|subcompact|24.714285714285715|
|   6|       suv|              18.5|
|   8|      null| 17.62857142857143|
|   8|   2seater|              24.8|
+----+----------+------------------+
only showing top 20 rows

